# Constraint Solving [B-KUL-H02A3A]: Project #1

Welcome to the first project in Constraint Solving.

For details on grading, academic integrity, grading, and submission, please see the Toledo assignment instructions.

Only `check2()` below makes up your grade; its score will be rescaled to 1 point and contribute that to your course grade.

## Warm-up (ungraded)
In the fist part of this project, we do a warm-up exercise to get acquainted with how to translate a natural language specification (i.e. the _spec_) of a combinatorial problem into a CPMpy model.
We also show how to use the provided checker to check and grade your model for _correctness_.
First, let's install the dependencies of the project.
Consider using a virtual Python environment.

In [1]:
%pip install matplotlib networkx --quiet
%pip install --upgrade cpmpy --quiet


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### Xander and Yara
Xander and Yara need to decide whom of them goes to fuel the car.
However, there is a restriction: the car cannot drive itself (yet), so either Xander or Yara (or both) will have to join the trip.

We have already created a model, two Boolean decision variables for who joins the car (`x` is true if-and-only-if (_iff_) Xander joins, `y` iff Yara joins), and a constraint to ensure at least one of them joins.
This is the 'stage 1' problem and model.

In [2]:
import cpmpy as cp
import project

modelP1 = cp.Model()
# the decision variables of this problem - do not change these or their names
x, y = cp.boolvar(2, name=["Xander", "Yara"])

modelP1 += [x | y]  # either Xander or Yara goes

/Users/karlalucic/Code/coursework/ConstraintSolving/myenv/lib/python3.13/site-packages/cpmpy/solvers/gurobi.py:46: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


#### Running the checker.

Let's call the `project.check1()` function on this model to check its correctness.

In [3]:
# let's check the correctness in this first part
project.check1(modelP1)

- STAGE1: PASS
- STAGE2: FAIL (Submission allows invalid assignment)
  Invalid part of the assignment:
Yara = False
Xander = True
Stages passed: 1/2



From the output, you should see that stage 1 was implemented correctly, but stage 2 not yet. Here is the stage 2 requirement:

- Stage 2: Xander got his driver license only recently, so he does not want to drive by himself. If he goes, Yara should join him.

Add a constraint that models this and thereby complete the warm-up exercise.

In [4]:
# we copy the model in-between cells, so that constraints from the previous run of this cell are removed in the next run
modelP1b = modelP1.copy()

# TODO: add stage 2
modelP1b += [x.implies(y)]  # if Xander goes, then Yara goes
project.check1(modelP1b)

- STAGE1: PASS
- STAGE2: PASS
Stages passed: 2/2



### How the checker works

The checker works by generating a limited set of various solutions from your model, and determining if each solution does not violate the constraints of each stage.
Each stage `i` is checked _independently_ of the other stages.
If you get stuck in a particular stage, you can move to the next stage and still obtain marks for that. Thus, you do not have to complete all stages perfectly to get a decent mark.

Because of the limit on the number of generated solutions, in exceptional cases it could be that your model passes a stage without being fully correct, because a faulty solution was not among the generated ones.

- You can influence how many solutions are generated with the `solution_limit` argument, or by setting a solving `time_limit` in seconds. Changing the defaults to lower means faster, but less reliable, checking.
- You can change the stages which are checked with the `stages` argument, either as e.g. `stages=2` for all stages up to and including 2, or as e.g. `stages={1,3,5}` to check only those individual stages.
- Note: use the decision variables as-is (e.g. do not modify their `name=...` parameter, or domain), since the checker relies on those variables being in your model.
    - However, you can (and will need to!) add new, auxiliary variables in your model. Everything will continue to work as expected.
    - Changing the checking code in `project.py` is not advisable, as we will use the provided version in the auto-grader.

If your answer is considered wrong, the checker will output a (minimal) partial assignment that is wrong.
If you are certain that your model adheres to the spec and that the assignment should be allowed, then you can share your notebook with the teaching team so we can verify it is indeed not a mistake on our side.
If the checker or other parts of the project fail in other unexpected ways, please let us know too.

## Project 1: POSIX-style Access Control Lists

Now the real, graded, project starts.

In Unix-based file systems, files are shared between multiple users.
To maintain organization and security, files have owners, permissions, sub-directories, protections, and so on.
Your goal will be to create a model which can generate the files and their permissions according to a set of rules concocted by security experts.

### Permissions and owners

Initially, our instance is defined by the following parameters:

- `n_files`: the number of files
- `n_users`: the number of users
- `n_perms`: the number of permission types, where `0 = read, 1 = write, 2 = execute`

And the decision variables are:

- `owners`: for each file, an integer variable designating which user owns it. For example, if `owners[2]` is assigned to `3`, then file `2` is owned by user `3`.
- `permissions`: for each file `f`, user `u`, and permission type `p`, the Boolean decision variable `permissions[f, u, p]` designates whether or not user `u` has permission `p` for file `f`. For example, if `permissions[2, 3, 0]` is assigned to `True`, then user `3` has permission type `0` (i.e. `read`) for file `2`, if `permissions[2, 3, 1]` is also true, then user `3` also has permission type `1` (i.e. `write`) permission for file 3, and so on.

In [ ]:
n_files = 6  # number of files
n_users = 4  # number of users
n_perms = 3  # number of permission types (0 = read, 1 = write, 2 = execute)

files = range(n_files)
users = range(n_users)
perms = range(n_perms)
read, write, execute = perms

# each file is owned by a user
owners = cp.intvar(0, n_users - 1, name="owner", shape=n_files)
# each file, user, and permission is enabled (True) or disabled (False)
permissions = cp.boolvar(name="permission", shape=(n_files, n_users, n_perms))

Model the following basic access rules:

- Stage 1:
  - The "admin" user (user 0) has `write` permissions for all files
  - The admin user owns file `0`
  - All users have read and write permissions for any files they own
- Stage 2:
  - All users with `execute` permission for a given file have `write` permissions for the same file
  - All users with `write` permission for a given file have `read` permissions for the same file

In [ ]:
model1 = cp.Model()

# stage 1
# user 0 (admin) has write (1) permission for all files
for f in files:
    model1 += [permissions[f, 0, write] ]

# user 0 (admin) owns file 0
model1 += [owners[0] == 0]

# all users have r+w permissions for any files they own
# if user u owns file f, then they must have r+w permission -> cond.implies(conequence)
for f in files:
    for u in users:
        model1 += [(owners[f] == u).implies(permissions[f, u, read])]
        model1 += [(owners[f] == u).implies(permissions[f, u, write])]


# stage 2
# execute -> write -> read
for f in files:
    for u in users:
        model1 += permissions[f, u, execute].implies(permissions[f, u, write])
        model1 += permissions[f, u, write].implies(permissions[f, u, read])

project.check2(model1, stages=2)

### Directories
Directories are files which contain other files.
This file structure influences the permissions.
The files can be seen as nodes in a graph, specifically a _directed tree_, with arcs (i.e. edges) from directories to the files they contain.
To keep track of which files contains which, we introduce Boolean variable `arcs[i,j]` for each file `i` and `j` which is true iff file `i` contains file `j`.

To visualize the graph (after calling `solve()` on the model), you can use the `project.draw_tree` function.

In [ ]:
model2 = model1.copy()

# `arcs[i,j]` is True iff file `i` contains file `j`
arcs = cp.boolvar(name="arcs", shape=(n_files, n_files))

model2.solve()

# draw tree (see its documentation for more information)
project.draw_tree(
    n_files,
    arcs,
    labels={i: f"{i}" for i in range(n_files)},
    node_color=["green" for i in range(n_files)],
)

Now model the following stages:

- Stage 3:
  - Files cannot have multiple parent directories. Thus, ensure that all files have at most one incoming edge.
  - Ensure that there is exactly one _root_ file, which has no incoming edges
- Stage 4:
  - Ensure that the files are part of a single directory structure, i.e. that the graph is connected
    - Hint: one way to model this is to ensure there is an edge between any two subsets of files
- Stage 5:
    - Each file inherits all (positive) permissions from its parent (i.e. containing) directory
    - Directories are non-executable
    - The first two users own one file each, the last two users own two files each
      - Hint: you can use a global constraint here

Note: If you run into an exception of the form `Exception: Non-Boolean argument '...' to '...'` for an argument you're sure is Boolean, try replacing the argument by the equivalent `... == True`.

In [ ]:
model3 = model2.copy()

#stage 3 
parent = cp.intvar(-1, n_files -1, name="parent", shape=n_files) # -1 is the root
is_root = cp.boolvar(name="is_root", shape=n_files)

# no self-parenting edges in the directory graph (diagonal entries of the adj matrix == False)
for f in files:
    model3 += arcs[f, f] == False

# only one root
model3 += [cp.sum(is_root) == 1]

# link "is_root" with "parent" and incoming edges (for consistent tree structure)
for child in files:
    incoming = cp.sum(arcs[:, child])   # counts how may parents the file has (should be 0 for root, or 1 otherwise)
    # each file is either root, or has exactly 1 incoming edge
    model3 += [is_root[child].implies(incoming == 0)]
    model3 += [(~is_root[child]).implies(incoming == 1)]
    # root has parent = -1
    model3 += [is_root[child] == (parent[child] == -1)]
    # a node can never be its own parent
    model3 += [parent[child] != child]
    # link parent variable with arcs
    for p_index in files:
        model3 += [arcs[p_index, child] == (parent[child] == p_index)]


#stage 4
depth = cp.intvar(0, n_files -1, name="depth", shape=n_files)

#connectivity via depth ordering
for f in files:
    #rooth has depth 0
    model3 += [is_root[f].implies(depth[f] == 0)]
    # non-root nodes have depth >= 1
    model3 += [(~is_root[f]).implies(depth[f] >= 1)]

#if there is an arc parent->child, then depth[child] = depth[parent] +1
for p_index in files:
    for c_index in files:
        model3 += arcs[p_index, c_index].implies(depth[c_index] == depth[p_index]+1)


#stage 5
is_directory = cp.boolvar(name="is_directory", shape=n_files)

# a file is a directoiry i if it has outgoing arcs
for f in files:
    has_child = cp.any([arcs[f, i] for i in files])
    model3 += [is_directory[f] == has_child]

# child inharents permissions form parent
for p_index in files:
    for c_index in files:
        for u in users:
            for p in perms:
                # either the paarent lack this perm or the child has it
                inherit = (~permissions[p_index, u, p]) | permissions[c_index, u, p]
                model3 += arcs[p_index, c_index].implies(inherit)

# directories are non executable
for f in files:
    for u in users:
        model3 += [is_directory[f].implies(~permissions[f, u, execute])]

# user0 owns 1 file, user1 1 file, user2 2 files, user3 2 files.
ownership = [1, 1, 2, 2]
model3 += [cp.GlobalCardinalityCount(owners, list(users), ownership)]

project.check2(model3, stages=5)


### Protected files and objective
File 3 and 4 are _protected_ files with special access rules.
Model the following specification:

- Stage 6:
  - All protected files must be in the same directory
  - Protected files cannot be directories
- Stage 7:
  - For auditing purposes, there must be at least one unprotected "log" file stored in the same directory as the protected files
  - The log file is not a directory
  - The log file must be readable by the admin user
  - The log file must be writeable by any user who owns one or more protected files
  - The protected files cannot be stored directly in the root directory
  - Each protected file must be owned by a different user

Note that the solution limit has been increased for the final check; you can change this number and the `time_limit` for faster checking during development.

In [ ]:
# IMPORTANT: this variable determines your grade. See submission instructions on Toledo for further details.
model = model3.copy()
protected = [3,4]

#stage 6
#both protected files have same parent
model += [parent[protected[0]] == parent[protected[1]]]

# protected files cannot be directories, but regular files
for f in protected:
    model += [~is_directory[f]]


# stage 7
is_log = cp.boolvar(name="is_log", shape=n_files)   # files that act as the auditing log
owns_protected = cp.boolvar(name="owns_protected", shape=n_users)

for f in protected:
    # protected files cannot be in root (must have a parent)
    model += [parent[f] != -1]
    #protected files are not log files
    model += [~is_log[f]]

# each protected file is owned by a different user
model += [owners[protected[0]] != owners[protected[1]]]

# at least one log file must exist
model += [cp.sum(is_log) >= 1]

# track which users own protected files
for u in users:
    ownes_any = cp.any(owners[p] == u for p in protected)
    model += [owns_protected[u] == ownes_any]

# log requierments
for f in files:
    #in the same directory as protected files (same parent)
    model += [is_log[f].implies(parent[f] == parent[protected[0]])]
    # log is not a directory
    model += [is_log[f].implies(~is_directory[f])]
    # readable by admin
    model += [is_log[f].implies(permissions[f, 0, read])]
    # witable by any user who owns >= 1 protected files
    for u in users:
        model += [(is_log[f] & owns_protected[u]).implies(permissions[f, u, write])]


project.check2(model, solution_limit=200)


Bonus stage:
  - Minimize the number of permissions granted (in order to maximize security and inconvenience)

In [ ]:
model_bonus = model.copy()

# TODO add bonus stage
model_bonus.minimize(cp.sum(permissions))